# Manage Coiled Resources

In [ ]:
import json
import os
import sys
from typing import Dict, List, Union

import boto3
import coiled
import polars as pl

In [ ]:
PROJ_ROOT = os.path.join(os.getcwd())

In [ ]:
_ = pl.Config.set_tbl_cols(None)
_ = pl.Config.set_fmt_str_lengths(1_000)
_ = pl.Config.set_tbl_width_chars(1_000)
_ = pl.Config.set_tbl_rows(None)

## About

Perform the following

1. Set up Coiled cloud provider (AWS)
   - create the following IAM entities
     - role
     - policy for EC2 instance
     - ongoing policy for IAM user
     - setup policy for IAM user
     - user
   - attach IAM policies to IAM role
     - policy for EC2 instance
     - policy to access S3 bucket
   - attach IAM policies to IAM user
     - ongoing policy
     - setup policy
   - create access key for IAM user
2. manage Coiled software environment
   - create
   - delete
   - list

## User Inputs

In [ ]:
coiled_account_name = "elsdes3"
coiled_se_name = "my-pip-env"
aws_region = 'us-east-2'
s3_bucket_name = 'oss-shared-scratchp'

In [ ]:
nb_dir_name = [dir_name for dir_name in os.listdir(os.path.join(PROJ_ROOT, 'code')) if dir_name != 'src'][0]
nb_dir_path = os.path.join(PROJ_ROOT, 'code', nb_dir_name)

In [ ]:
coiled_iam_user = {
    'user_path': "/",
    'user_name': 'demo',
}
coiled_iam_role = {
    "role_path": "/",
    "role_name": "coiled-elsdes3",
    "role_trust_policy": json.dumps(
        {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Sid": "",
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "ec2.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        }
    ),
    "role_description": "A role created by Coiled to be attached to EC2 instances."
}
coiled_iam_instance_policy = {
    'policy_name': "CoiledInstancePolicy",
    'policy_description': "Policy to allow coiled managed instances to submit logs to cloudwatch",
    'policy_tags': [{"Key": "Name", "Value": 'coiled-instance-policy'}],
    'policy_document': json.dumps(
        {
            "Version": "2012-10-17",
                "Statement": [
                    {
                        "Sid": "CoiledEC2Policy",
                        "Effect": "Allow",
                        "Action": [
                            "logs:CreateLogGroup",
                            "logs:CreateLogStream",
                            "logs:PutLogEvents"
                        ],
                        "Resource": "*"
                    }
                ]
        }
    )
}
coiled_iam_setup_policy = {
    'policy_name': "demo-setup",
    'policy_description': "Coiled setup IAM policy",
    'policy_tags': [{"Key": "Name", "Value": 'coiled-setup-iam-policy'}],
    "policy_document": json.dumps(
        {
            "Statement": [
                {
                    "Sid": "Setup",
                    "Effect": "Allow",
                    "Resource": "*",
                    "Action": [
                        "ec2:AssociateRouteTable",
                        "ec2:AttachInternetGateway",
                        "ec2:CreateInternetGateway",
                        "ec2:CreateRoute",
                        "ec2:CreateRouteTable",
                        "ec2:CreateSubnet",
                        "ec2:CreateVpc",
                        "ec2:DeleteInternetGateway",
                        "ec2:DeleteRoute",
                        "ec2:DeleteRouteTable",
                        "ec2:DeleteSubnet",
                        "ec2:DeleteVpc",
                        "ec2:DescribeInternetGateways",
                        "ec2:DetachInternetGateway",
                        "ec2:DisassociateRouteTable",
                        "ec2:ModifySubnetAttribute",
                        "ec2:ModifyVpcAttribute",
                        "iam:AddRoleToInstanceProfile",
                        "iam:AttachRolePolicy",
                        "iam:CreateRole",
                        "iam:CreatePolicy",
                        "iam:CreateServiceLinkedRole",
                        "iam:CreateInstanceProfile",
                        "iam:DeleteRole",
                        "iam:ListPolicies",
                        "iam:ListInstanceProfiles",
                        "iam:ListAttachedRolePolicies",
                        "iam:TagInstanceProfile",
                        "iam:TagPolicy",
                        "iam:TagRole"
                    ]
                }
            ],
            "Version": "2012-10-17"
        }
    )
}
coiled_iam_ongoing_policy = {
    'policy_name': "demo-ongoing",
    'policy_description': "Coiled ongoing IAM policy",
    'policy_tags': [{"Key": "Name", "Value": 'coiled-ongoing-iam-policy'}],
    "policy_document": json.dumps(
        {
            "Statement": [
                {
                    "Sid": "Ongoing",
                    "Effect": "Allow",
                    "Resource": "*",
                    "Action": [
                        "ec2:AuthorizeSecurityGroupIngress",
                        "ec2:CreateFleet",
                        "ec2:CreateLaunchTemplate",
                        "ec2:CreateLaunchTemplateVersion",
                        "ec2:CreateRoute",
                        "ec2:CreateSecurityGroup",
                        "ec2:CreateTags",
                        "ec2:DeleteLaunchTemplate",
                        "ec2:DeleteLaunchTemplateVersions",
                        "ec2:DeleteSecurityGroup",
                        "ec2:DescribeAvailabilityZones",
                        "ec2:DescribeImages",
                        "ec2:DescribeInstances",
                        "ec2:DescribeInstanceTypeOfferings",
                        "ec2:DescribeInstanceTypes",
                        "ec2:DescribeInternetGateways",
                        "ec2:DescribeLaunchTemplates",
                        "ec2:DescribeRegions",
                        "ec2:DescribeRouteTables",
                        "ec2:DescribeSecurityGroups",
                        "ec2:DescribeSubnets",
                        "ec2:DescribeVpcs",
                        "ec2:RunInstances",
                        "ec2:TerminateInstances",
                        "ecr:BatchCheckLayerAvailability",
                        "ecr:BatchGetImage",
                        "ecr:GetAuthorizationToken",
                        "ecr:GetDownloadUrlForLayer",
                        "iam:GetInstanceProfile",
                        "iam:GetRole",
                        "iam:ListPolicies",
                        "iam:PassRole",
                        "iam:TagRole",
                        "logs:CreateLogGroup",
                        "logs:CreateLogStream",
                        "logs:DescribeLogGroups",
                        "logs:DescribeLogStreams",
                        # "logs:FilterLogEvents",
                        "logs:GetLogEvents",
                        "logs:PutLogEvents",
                        "logs:PutRetentionPolicy",
                        "logs:TagLogGroup",
                        "logs:TagResource",
                        "sts:GetCallerIdentity"
                    ]
                }
            ],
            "Version": "2012-10-17"
        }
    )
}

Define helper functions to manage cloud (AWS) resources

In [ ]:
def create_iam_role(
    role_path: str,
    role_name: str,
    role_description: str,
    role_trust_policy: Dict,
    aws_region: str,
) -> None:
    """Create IAM Role."""
    iam_client = boto3.client("iam", region_name=aws_region)
    role_creation_response = iam_client.create_role(
        Path=role_path,
        RoleName=role_name,
        AssumeRolePolicyDocument=role_trust_policy,
        Description=role_description,
        MaxSessionDuration=3600,
    )
    try:
        http_code = role_creation_response["ResponseMetadata"][
            "HTTPStatusCode"
        ]
        assert http_code == 200
        print(f"Created IAM role {role_name} successfully")
    except AssertionError as e:
        print(
            "Could not create the IAM role "
            f"{role_name} successfully\n{str(e)}"
        )


def create_iam_policy(
    aws_region: str,
    policy_name: str,
    policy_document: Dict,
    policy_description: str,
    policy_tags: List[Dict],
) -> None:
    """Create an IAM policy."""
    iam_client = boto3.client("iam", region_name=aws_region)
    policy_creation_response = iam_client.create_policy(
        PolicyName=policy_name,
        PolicyDocument=policy_document,
        Description=policy_description,
        Tags=policy_tags,
    )
    try:
        http_code = policy_creation_response["ResponseMetadata"][
            "HTTPStatusCode"
        ]
        assert http_code == 200
        print(f"Created IAM policy {policy_name}")
    except AssertionError as e:
        print(
            "Could not create IAM policy "
            f"{policy_name} successfully\n{str(e)}"
        )


def create_iam_user(user_path: str, user_name: str, aws_region: str) -> None:
    """Create IAM User."""
    iam_client = boto3.client("iam", region_name=aws_region)
    user_creation_response = iam_client.create_user(
        Path=user_path,
        UserName=user_name,
    )
    try:
        http_code = user_creation_response["ResponseMetadata"][
            "HTTPStatusCode"
        ]
        assert http_code == 200
        print(f"Created IAM role {user_name} successfully")
    except AssertionError as e:
        print(
            "Could not create the IAM user "
            f"{user_name} successfully\n{str(e)}"
        )


def attach_iam_policy_to_iam_role(
    policy_name: str, iam_role_name: str, aws_region: str, max_items: int=500
)  -> None:
    """Attach an IAM policy to an IAM role."""
    iam_client = boto3.client("iam", region_name=aws_region)
    iam_policy = [
        policy
        for policy in iam_client.list_policies(
            Scope="All", OnlyAttached=False, MaxItems=max_items
        )["Policies"]
        if policy['PolicyName'] == policy_name
    ][0]
    
    iam_policy_arn = iam_policy['Arn']
    policy_attachment_response = iam_client.attach_role_policy(
        RoleName=iam_role_name, PolicyArn=iam_policy_arn
    )
    try:
        http_code = policy_attachment_response["ResponseMetadata"][
            "HTTPStatusCode"
        ]
        assert http_code == 200
        print(
            f"Attached IAM policy {iam_policy_arn} to role {iam_role_name} "
            "successfully"
        )
    except AssertionError as e:
        print(
            f"Could not attach IAM policy {iam_policy_arn} to role "
            f"{iam_role_name} successfully\n{str(e)}"
        )

def attach_iam_policy_to_iam_user(
    policy_name: str, iam_user_name: str, aws_region: str, max_items: int=500
)  -> None:
    """Attach an IAM policy to an IAM role."""
    iam_client = boto3.client("iam", region_name=aws_region)
    iam_policy = [
        policy
        for policy in iam_client.list_policies(
            Scope="All", OnlyAttached=False, MaxItems=max_items
        )["Policies"]
        if policy['PolicyName'] == policy_name
    ][0]
    
    iam_policy_arn = iam_policy['Arn']
    policy_attachment_response = iam_client.attach_user_policy(
        UserName=iam_user_name, PolicyArn=iam_policy_arn
    )
    try:
        http_code = policy_attachment_response["ResponseMetadata"][
            "HTTPStatusCode"
        ]
        assert http_code == 200
        print(
            f"Attached IAM policy {iam_policy_arn} to user {iam_user_name} "
            "successfully"
        )
    except AssertionError as e:
        print(
            f"Could not attach IAM policy {iam_policy_arn} to user "
            f"{iam_user_name} successfully\n{str(e)}"
        )


def create_user_access_key(iam_user_name: str, aws_region: str) -> None:
    """Create IAM access key for user."""
    iam_client = boto3.client("iam", region_name=aws_region)
    key_creation_response = iam_client.create_access_key(
        UserName=iam_user_name,
    )
    try:
        http_code = key_creation_response["ResponseMetadata"][
            "HTTPStatusCode"
        ]
        assert http_code == 200
        print(
            f"Created IAM access key for user {iam_user_name} "
            "successfully"
        )
    except AssertionError as e:
        print(
            f"Could not create IAM access key for user "
            f"{iam_user_name} successfully\n{str(e)}"
        )

Define helper functions to manage Coiled resources

In [ ]:
def list_coiled_software_envs() -> List[Dict[str, Union[int, str]]]:
    """Get list of all Coiled software environments and installed packages."""
    dfs_se = []
    for k, v in coiled.list_software_environments().items():
        se_info = coiled.get_software_info(k)
        latest_spec = v['latest_spec']
        latest_build = latest_spec['latest_build']
        state, reason, gpu, latest_build_id, latest_spec_id = [
            latest_build['state'],
            latest_build['reason'],
            latest_spec['gpu_enabled'],
            latest_build['id'],
            latest_spec['id'],
        ]
        se_dict = dict(
            name=k,
            alias_id=v['id'],
            state=state,
            reason=reason,
            gpu_enabled=gpu,
            id=latest_spec_id,
            created=latest_spec['created'],
            updated=latest_spec['updated'],
            container_uri=latest_spec['container_uri'],
            architecture=latest_spec['architecture'],
            latest_build_id=latest_build_id,
        )
        info_dict = {
            k:v
            for k,v in coiled.get_software_info(se_record['name']).items()
            if k in [
                'id',
                'alias_id',
                'gpu_enabled',
                'latest_build_id',
                'created',
                'updated',
                'container_uri',
                'architecture',
                'raw_pip',
            ]
        }
        se_dict.update(info_dict)
        dfs_se.append(
            pl.DataFrame(se_dict)
            .with_columns([pl.lit(True).alias('in_software_env')])
        )
    df = pl.concat(dfs_se)
    return df

## Setup Cloud Storage (AWS)

In [ ]:
def change_s3_bucket_state(
    s3_bucket_name: str,
    aws_region: str = "us-east-2",
    acl: str = "private",
    state: str = "present",
) -> None:
    """Create S3 bucket."""
    s3_client = boto3.client("s3", region_name=aws_region)
    bucket_names = [b["Name"] for b in s3_client.list_buckets()["Buckets"]]
    if state == "present":
        if s3_bucket_name not in bucket_names:
            bucket_state_change_response = s3_client.create_bucket(
                ACL=acl,
                Bucket=s3_bucket_name,
                CreateBucketConfiguration={"LocationConstraint": aws_region},
            )
            try:
                assert (
                    bucket_state_change_response["ResponseMetadata"][
                        "HTTPStatusCode"
                    ]
                    == 200
                )
                print(f"Bucket {s3_bucket_name} created successfully")
            except AssertionError as e:
                print(
                    f"Bucket {s3_bucket_name} not created successfully. "
                    f"Got error message:\n{e}"
                )
        else:
            print(f"Found existing bucket {s3_bucket_name}")
    else:
        if s3_bucket_name not in bucket_names:
            bucket_state_change_response = s3_client.delete_bucket(
                Bucket=s3_bucket_name
            )
        else:
            print(f"Bucket {s3_bucket_name} does not exist")


def block_public_access_to_s3_bucket(
    s3_bucket_name: str,
    aws_region: str = "us-east-2",
) -> None:
    """Block access to S3 bucket."""
    s3_client = boto3.client("s3", region_name=aws_region)
    set_public_access_response = s3_client.put_public_access_block(
        Bucket=s3_bucket_name,
        PublicAccessBlockConfiguration={
            "BlockPublicAcls": True,
            "IgnorePublicAcls": True,
            "BlockPublicPolicy": True,
            "RestrictPublicBuckets": True,
        },
    )
    try:
        http_status_code = set_public_access_response["ResponseMetadata"][
            "HTTPStatusCode"
        ]
        assert http_status_code == 200
        print(f"Bucket {s3_bucket_name} access blocked successfully")
    except AssertionError as e:
        print(
            f"Bucket {s3_bucket_name} access was not successfully blocked. "
            f"Got error message:\n{e}"
        )

In [ ]:
%%time
change_s3_bucket_state(s3_bucket_name, aws_region, "private", "present")

In [ ]:
%%time
block_public_access_to_s3_bucket(s3_bucket_name, aws_region)

## Setup Coiled Cloud Provider (AWS)

### Create IAM Role

In [ ]:
%%time
create_iam_role(aws_region=aws_region, **coiled_iam_role)

### Create IAM Policies

Coiled instance policy

In [ ]:
%%time
create_iam_policy(aws_region=aws_region, **coiled_iam_instance_policy)

Coiled setup policy

In [ ]:
%%time
create_iam_policy(aws_region=aws_region, **coiled_iam_setup_policy)

Coiled ongoing policy

In [ ]:
%%time
create_iam_policy(aws_region=aws_region, **coiled_iam_ongoing_policy)

### Create IAM User

In [ ]:
%%time
create_iam_user(aws_region=aws_region, **coiled_iam_user)

### Attach IAM Policies to IAM Role

Coiled instance policy

In [ ]:
%%time
attach_iam_policy_to_iam_role(
    policy_name=coiled_iam_instance_policy['policy_name'],
    iam_role_name=coiled_iam_role['role_name'],
    aws_region=aws_region,
    max_items=123,
)

Access to S3 bucket

In [ ]:
%%time
attach_iam_policy_to_iam_role(
    policy_name='AmazonS3FullAccess',
    iam_role_name=coiled_iam_role['role_name'],
    aws_region=aws_region,
    max_items=1000,
)

### Attach IAM Policies to IAM User

Coiled setup policy

In [ ]:
%%time
attach_iam_policy_to_iam_user(
    policy_name=coiled_iam_setup_policy['policy_name'],
    iam_user_name=coiled_iam_user['user_name'],
    aws_region=aws_region,
    max_items=123,
)

Coiled ongoing policy

In [ ]:
%%time
attach_iam_policy_to_iam_user(
    policy_name=coiled_iam_ongoing_policy['policy_name'],
    iam_user_name=coiled_iam_user['user_name'],
    aws_region=aws_region,
    max_items=123,
)

### Create IAM Access Key for IAM User

In [ ]:
%%time
create_user_access_key(coiled_iam_user['user_name'], aws_region)

**Notes**

1. This step is ignored by `coiled setup aws ...` which creates its own access key for the IAM user.

## Manage Coiled Software Environment(s)

### List

Get list of software environments and installed packages

In [ ]:
%%time
list_coiled_software_envs()

### Create

In [ ]:
%%time
coiled.create_software_environment(
    name=coiled_se_name,
    pip="requirements_coiled.txt",
)

### List

Get list of software environments and installed packages

In [ ]:
%%time
df_se = list_coiled_software_envs()
df_se

For single software environment, compare installed packages to those in `requirements.txt`

In [ ]:
%%time
df_se_reqs = (
    pl.read_csv(
        os.path.join(nb_dir_path, 'requirements.txt'),
        has_header=False,
        new_columns=['package'],
    )
    .with_columns(
        [
            pl.lit(coiled_se_name).alias('name'),
            pl.lit(True).alias('in_requirements_txt')
        ]
    )
    .join(
        df_se,
        left_on=['name', 'package'],
        right_on=['name', 'raw_pip'],
        how='outer',
    )
    .with_columns(
        [
            pl.col('in_software_env').fill_null(False),
            pl.col('in_requirements_txt').fill_null(False),
        ]
    )
)
df_se_reqs

### Delete

In [ ]:
coiled.delete_software_environment(name=coiled_se_name)

### List

Get list of software environments and installed packages

In [ ]:
%%time
list_coiled_software_envs()

## Links

1. `boto3`
   - [Manage IAM Access Keys](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/iam-example-managing-access-keys.html)
2. Coiled
   - Set up Coiled cloud provider
     - Automatic setup
       - [full docs](https://docs.coiled.io/user_guide/setup/aws/cli.html#automatic-setup)
       - [keyword arguments](https://docs.coiled.io/user_guide/setup/aws/cli.html#coiled-setup-aws)
     - Manual setup
       - [full docs](https://docs.coiled.io/user_guide/setup/aws/manual.html#manual-setup)
       - [required IAM policies to be assigned to IAM user](https://docs.coiled.io/user_guide/setup/aws/manual.html#create-iam-policies)
   - Manage Coiled software environment(s)
     - [full docs](https://docs.coiled.io/user_guide/software/manual.html#manual-software-environments)